# GOALS
To create a GBQ view containing a daily state on users engagement.
- generate dates index
- generate users and dates index
- fill last_login_date for each date/user key
- fill last_consumption_start_date for each date/user key
- fill last_consumption_completion_date for each date/user key

# PACKAGES

In [1]:
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
import logging

# PARAMETERS

In [2]:
logger = logging.getLogger("pandas_gbq")
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

project_id = "analytics-dev-308300"

credentials = service_account.Credentials.from_service_account_file(
    "../keys/gcp_key.json",
)

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# FUNCTIONS

## get data

In [3]:
def get_data():
    """
    ()-->df
    """
    query = """
        SELECT DISTINCT
            user_id, created_at, extraction_date,
            CONCAT(first_name, ' ', last_name) as user_name
        FROM
            dtm_engagement.dim_users
        WHERE
            group_id=1818
    """
    users_df = pd.read_gbq(
        query=query, credentials=credentials, project_id=project_id)

    users_lst = users_df['user_id'].tolist()

    
    creation_df= users_df[['user_id','user_name','created_at']].drop_duplicates(ignore_index=True)
    creation_df['created_at']=pd.to_datetime(
        creation_df['created_at'], utc=True
        )

    query = """
        SELECT DISTINCT
            user_id, last_login, extraction_date
        FROM
            dtm_engagement.hist_users
        WHERE
            group_id=1818
        ORDER BY
            extraction_date DESC
        """

    login_df = pd.read_gbq(
        query=query, credentials=credentials, project_id=project_id)

    login_df=login_df.drop_duplicates(subset=['user_id','extraction_date'],keep='last',ignore_index=True)
    
    login_df["last_login"] = pd.to_datetime(
        login_df["last_login"], utc=True
    )

    query = """
        SELECT DISTINCT
            user_id,
            set_id,
            started_at,
            completed_at
        FROM
            dtm_engagement.ft_content_consumption
        WHERE
            group_id=1818
        """
    consumption_df = pd.read_gbq(
        query=query, credentials=credentials, project_id=project_id
    )

    consumption_df["started_at"] = pd.to_datetime(
        consumption_df["started_at"], utc=True
    )
    consumption_df["completed_at"] = pd.to_datetime(
        consumption_df["completed_at"], utc=True
    )

    return users_lst, creation_df, login_df, consumption_df

## generate base (users and dates) data frame

In [4]:
def create_base_df(
    ls_users,
    creation_df,
    start_date="2021-08-16",
    end_date=pd.Timestamp.today().strftime("%Y-%m-%d"),
):
    """
    (date-like, date_like, series) --> df
    Create a dataframe with one row for each combination of user and date. Date range is defined by start_date and end_date (excluded).
    """
    dates_index = (
        pd.to_datetime(
            pd.date_range(start=start_date, end=end_date, name="action_date")
        )
        .strftime("%Y-%m-%d")
        .to_list()
    )

    actions_dict = [
        {"action_date": action_date, "user_id": user}
        for action_date in dates_index
        for user in ls_users
    ]
    
    base_df=pd.DataFrame(actions_dict)
    base_df=base_df.merge(creation_df, how='left',on='user_id')
    base_df=base_df.drop(index=base_df[base_df['created_at']>base_df['action_date']].index)
    
    return base_df

## fill max date of interest for each date/user key

In [5]:
def max_date(consumption_df, reporting_date, user_id, date_of_interest):
    """
    (df,date like str, int)
    Select the maximum value for date_of_interest field that is inferior to the reporting date (23:59:59), for the specified user_id.
    """
    max_start = consumption_df[
        (consumption_df["user_id"] == user_id)
        & (
            consumption_df[date_of_interest]
            <= pd.Timestamp(reporting_date + " 23:59:59", tz="UTC")
        )
    ][date_of_interest].max()

    return max_start

## Fill the number of completed sets by period

In [6]:
def calculate_nb_of_sets_of_interest(consumption_df,reporting_date,user_id,date_of_interest,nb_of_days):
    """
    (df, date like str, int, int, date like str)-->number
    For the user_id, count the number of set_ids where completed at is between reporting_date-number_of_days and reporting_date.
    """

    number_of_sets = consumption_df[(consumption_df['user_id']==user_id) & 
                                    (consumption_df[date_of_interest].between(pd.Timestamp(reporting_date + " 23:59:59", tz="UTC")-pd.Timedelta(nb_of_days,'days'),pd.Timestamp(reporting_date + " 23:59:59", tz="UTC")))]['set_id'].count()
    
    return number_of_sets

## Set user status based on dates

In [15]:
def user_status(timedelta_since_last_login, timedelta_since_last_start,timedelta_since_last_completion,ever_logged):
    """
    (timedelta,timedelta,timedelta,bool)--> str
    """
    if not ever_logged:
        return '0.bird'
    elif timedelta_since_last_completion <= pd.Timedelta(7,'D'):
        return '4.learner'
    elif timedelta_since_last_start <= pd.Timedelta(7,'D'):
        return '3.consumer'
    elif timedelta_since_last_login <= pd.Timedelta(7,'D'):
        return '2.curious'
    else:
        return '1.missing'

assert user_status(
    timedelta_since_last_login=pd.Timedelta(pd.NaT),
    timedelta_since_last_start=pd.Timedelta('2 days 13:41:36'),
    timedelta_since_last_completion=pd.Timedelta('1 days 11:29:23'),
    ever_logged=True)=='4.learner'

assert user_status(
    timedelta_since_last_login=pd.Timedelta(pd.NaT),
    timedelta_since_last_start=pd.Timedelta('1 days 13:41:36'),
    timedelta_since_last_completion=pd.Timedelta('7 days 11:29:23'),
    ever_logged=True)=='3.consumer'

assert user_status(
    timedelta_since_last_login=pd.Timedelta(pd.NaT),
    timedelta_since_last_start=pd.Timedelta('1 days 13:41:36'),
    timedelta_since_last_completion=pd.Timedelta(pd.NaT),
    ever_logged=True)=='3.consumer'

assert user_status(
    timedelta_since_last_login=pd.Timedelta('1 days 13:41:36'),
    timedelta_since_last_start=pd.Timedelta('8 days 13:41:36'),
    timedelta_since_last_completion=pd.Timedelta('7 days 11:29:23'),
    ever_logged=True)=='2.curious'

assert user_status(
    timedelta_since_last_login=pd.Timedelta('7 days 13:41:36'),
    timedelta_since_last_start=pd.Timedelta('8 days 13:41:36'),
    timedelta_since_last_completion=pd.Timedelta('10 days 11:29:23'),
    ever_logged=True)=='1.missing'

assert user_status(
    timedelta_since_last_login=pd.Timedelta('0 days'),
    timedelta_since_last_start=pd.Timedelta('0 days'),
    timedelta_since_last_completion=pd.Timedelta('0 days'),
    ever_logged=False)=='0.bird'

## create engagement df by completing base_df with calculated fields

In [8]:
def generate_engagement_df(base_df,consumption_df,login_df):
    """
    (df,df)-->df
    Update the start_date for each user with the maximum value inferior to the action_date. Action date is extended with 23:59:59 to encompass the entire day.
    """
    engagement_df=base_df.copy()

    engagement_df['ever_logged']=engagement_df.apply(lambda x: max_date(login_df,x['action_date'],x['user_id'],'last_login'), axis=1)
    
    engagement_df['last_login_date']=engagement_df.apply(lambda x: max_date(login_df,x['action_date'],x['user_id'],'last_login'), axis=1)
    engagement_df['timedelta_since_last_login']=pd.to_datetime(engagement_df['action_date']+' 23:59:59',utc=True)-pd.to_datetime(engagement_df['last_login_date'],utc=True)
    engagement_df['days_since_last_login']=engagement_df['timedelta_since_last_login'].dt.days
    
    engagement_df['last_start_date']=engagement_df.apply(lambda x: max_date(consumption_df,x['action_date'],x['user_id'],'started_at'), axis=1)
    engagement_df['timedelta_since_last_start']=pd.to_datetime(engagement_df['action_date']+' 23:59:59',utc=True)-engagement_df['last_start_date']
    engagement_df['days_since_last_start']=engagement_df['timedelta_since_last_start'].dt.days
    
    engagement_df['last_completion_date']=engagement_df.apply(lambda x: max_date(consumption_df,x['action_date'],x['user_id'],'completed_at'), axis=1)
    engagement_df['timedelta_since_last_completion']=pd.to_datetime(engagement_df['action_date']+' 23:59:59',utc=True)-engagement_df['last_completion_date']
    engagement_df['days_since_last_completion']=engagement_df['timedelta_since_last_completion'].dt.days
    
    engagement_df['nb_of_completed_sets']=engagement_df.apply(lambda x: calculate_nb_of_sets_of_interest(consumption_df=consumption_df,reporting_date=x['action_date'],user_id=x['user_id'],date_of_interest='completed_at',nb_of_days=7),axis=1)
    
    engagement_df['user_status']=engagement_df.apply(lambda x:
                                                               user_status(
                                                                   x['timedelta_since_last_login'],
                                                                   x['timedelta_since_last_start'],
                                                                   x['timedelta_since_last_completion'],
                                                                   ever_logged=True),
                                                               axis=1
                                                              )
    
    return engagement_df

# DATA WRANGLING

In [9]:
ls_users, df_creation, df_login, df_consumption = get_data()

Requesting query... 
Query running...
Job ID: 89440b9e-0478-4511-abd9-8315e75505f6
Query done.
Processed: 400.4 KB Billed: 10.0 MB
Standard price: $0.00 USD

Got 45 rows.

Requesting query... 
Query running...
Job ID: bdac28d8-c0f3-4b81-b596-097c47670548
Query done.
Processed: 107.8 KB Billed: 10.0 MB
Standard price: $0.00 USD

Got 625 rows.

Requesting query... 
Query running...
Job ID: d31920cc-d28e-458c-8829-70952d845f41
Query done.
Processed: 23.0 KB Billed: 20.0 MB
Standard price: $0.00 USD

Got 330 rows.



In [10]:
df_engagement = generate_engagement_df(create_base_df(ls_users,df_creation),df_consumption,df_login)
df_engagement

,action_date,user_id,user_name,created_at,last_login_date,timedelta_since_last_login,days_since_last_login,last_start_date,timedelta_since_last_start,days_since_last_start,last_completion_date,timedelta_since_last_completion,days_since_last_completion,nb_of_completed_sets,user_status
0,2021-08-16,20029,JM Benedetto,2021-07-25 00:11:04+00:00,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
1,2021-08-16,20030,Aurelien Jacomy,2021-07-25 02:42:03+00:00,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
2,2021-08-16,20032,Martin Ciriani,2021-07-25 21:00:47+00:00,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
7,2021-08-16,20082,H Aluno,2021-07-31 23:54:33+00:00,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
8,2021-08-16,20100,Luana Amarante,2021-08-03 11:05:51+00:00,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,2021-09-05,20165,Jose Carlos Pasquini Catozich,2021-08-12 11:33:06+00:00,2021-08-18 13:56:50+00:00,18 days 10:03:09,18.0,2021-08-30 16:25:35+00:00,6 days 07:34:24,6.0,2021-09-04 21:58:58+00:00,1 days 02:01:01,1.0,3,4.learner
941,2021-09-05,20118,Martin Ciriani,2021-08-06 12:29:27+00:00,2021-08-20 20:01:05+00:00,16 days 03:58:54,16.0,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
942,2021-09-05,20293,Ricardo CRKS,2021-08-25 13:15:49+00:00,2021-08-25 20:38:20+00:00,11 days 03:21:39,11.0,2021-08-30 20:08:42+00:00,6 days 03:51:17,6.0,2021-09-02 20:29:00+00:00,3 days 03:30:59,3.0,3,4.learner
943,2021-09-05,20138,Sergio Silva,2021-08-09 17:03:28+00:00,2021-08-26 11:06:44+00:00,10 days 12:53:15,10.0,2021-09-03 19:48:04+00:00,2 days 04:11:55,2.0,2021-09-03 19:50:02+00:00,2 days 04:09:57,2.0,3,4.learner


In [11]:
df_engagement.to_gbq('raw_engagement.users_engagement',project_id=project_id,if_exists='replace',credentials=credentials)

884 out of 884 rows loaded.
1it [00:04,  4.43s/it]


In [23]:
df_login.head(5)

,user_id,last_login,extraction_date
0,20029,NaT,2021-09-05
1,20030,NaT,2021-09-05
2,20032,NaT,2021-09-05
3,20294,NaT,2021-09-05
4,20295,NaT,2021-09-05


In [40]:
df_log['loggin_1st']=df_log.groupby(by=['user_id'])['last_login'].transform(min)

In [59]:
df_log[df_log['user_id']==20165]

,extraction_date,user_id,last_login,loggin_1st
390,2021-08-24,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
391,2021-08-25,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
392,2021-08-26,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
393,2021-08-27,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
394,2021-08-28,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
395,2021-08-29,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
396,2021-08-30,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
397,2021-08-31,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
398,2021-09-01,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00
399,2021-09-02,20165,2021-08-18 13:56:50+00:00,2021-08-18 13:56:50+00:00


In [57]:
df_login['extraction_date'].max()

'2021-09-05'

In [58]:
df_login['last_login'].max()

Timestamp('2021-08-29 21:45:15+0000', tz='UTC')